In [15]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from estados import estados_brasileiros
from functools import reduce
import unicodedata
from pyspark.sql.functions import col

In [3]:
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
# conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f79ba3a8-18b9-4ee7-9a22-b53d246bfe88;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 448ms :: artifacts dl 16ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------

In [22]:
dfTweets = spark.read.csv("s3a://cf-murilo-pi-2022-01-sptech-bucket-bruto/tweets-analisados-2022-06-07.csv", header=True,
                          inferSchema=True)

In [23]:
dfTweetsTratado = dfTweets.filter(dfTweets.len > 5)

In [24]:
dfTweetsTratado.coalesce(1).write.mode('overwrite').option("header", True).csv("s3a://cf-murilo-pi-2022-01-sptech-bucket-tratados/tweets", emptyValue='')

22/06/08 01:49:46 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/08 01:49:46 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [25]:
dfEnem = spark.read.option('delimiter', ',').option("encoding", "ISO-8859-1").csv("s3a://cf-murilo-pi-2022-01-sptech-bucket-bruto/enem-sample-bruto.csv",
                                                                                  header=True, inferSchema=True)

In [12]:
# Criar Sample
# dfEnemSample = dfEnem.sample(0.1, seed=69420)

In [13]:
# dfEnemSample.coalesce(1).write.option("header",True).csv("./tmp/spark_output/datacsv/enem", emptyValue='')

In [26]:
dfEnemEstadosTratados = dfEnem.na.replace(to_replace=estados_brasileiros)

In [27]:
dfEnemTratado = dfEnemEstadosTratados.selectExpr("IN_TREINEIRO as Treineiro",
                                                 "SG_UF_PROVA as Estado",
                                                 "TP_PRESENCA_CN as Presenca_CN",
                                                 "TP_PRESENCA_CH as Presenca_CH",
                                                 "TP_PRESENCA_LC as Presenca_LC",
                                                 "TP_PRESENCA_MT as Presenca_MT",
                                                 "NU_NOTA_CN as Nota_CN",
                                                 "NU_NOTA_CH as Nota_CH",
                                                 "NU_NOTA_LC as Nota_LC",
                                                 "NU_NOTA_MT as Nota_MT",
                                                 "TP_STATUS_REDACAO as Status_Redacao",
                                                 "NU_NOTA_REDACAO as Nota_Redacao")

In [28]:
dfEnemTratado.coalesce(1).write.option("header", True).csv("s3a://cf-murilo-pi-2022-01-sptech-bucket-tratados/enem", emptyValue='')

22/06/08 01:50:07 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/08 01:50:08 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [47]:
# dfEnemTratadoSample = dfEnemTratado.sample(0.1, seed=69420)

In [48]:
# dfEnemTratadoSample.coalesce(1).write.option("header", True).csv("./tmp/spark_output/datacsv/enem", emptyValue='')

In [29]:
dfIDD = spark.read.json("s3a://cf-murilo-pi-2022-01-sptech-bucket-bruto/IDD_2019.json", multiLine=True)
dfIDD = dfIDD.fillna(0) # This is the number 0
dfIDDEstadosTratados = dfIDD.na.replace(to_replace=estados_brasileiros)
dfIDDEstadosTratados.show(vertical=True)

-RECORD 0-----------------------------------------------------------------------
 Ano                                                     | 2019                 
 Categoria Administrativa                                | Privada com fins ... 
 Código da IES                                           | 1414                 
 Código da Área                                          | 28                   
 Código do Curso                                         | 45877                
 Código do Município                                     | 1100205              
 IDD (Contínuo)                                          | 3,2835               
 IDD (Faixa)                                             | 4                    
 Modalidade de Ensino                                    | Educação Presencial  
 Município do Curso                                      | Porto Velho          
 Nome da IES                                             | CENTRO UNIVERSITÁ... 
 Nota Bruta - IDD           

In [30]:
colunas = dfIDDEstadosTratados.columns
novas_colunas = list(map(lambda item: unicodedata.normalize('NFKD', item.replace("(", "").replace(")", "").replace("-","").replace(
    " ", "_").upper()).encode('ascii', 'ignore').decode('utf-8', 'ignore'), colunas))
dfIDDComNovasColunas = reduce(lambda data, idx: data.withColumnRenamed(colunas[idx], novas_colunas[idx]),
                              range(len(colunas)), dfIDDEstadosTratados)

In [31]:
dfIDDTratado = dfIDDComNovasColunas.selectExpr("IDD_CONTINUO as IDD_Continuo",
                                                           "IDD_FAIXA as IDD_Faixa",
                                                           "NOTA_BRUTA__IDD as Nota_Bruta_IDD",
                                                           "SIGLA_DA_UF as Estado_Curso",
                                                           "NOME_DA_IES as Instituicao_Ensino",
                                                           "No_DE_CONCLUINTES_INSCRITOS as Concluintes_Inscritos",
                                                           "No_DE_CONCLUINTES_PARTICIPANTES as Concluintes_Participantes",
                                                           "No_DE_CONCLUINTES_PARTICIPANTES_COM_NOTA_NO_ENEM as Concluintes_Participantes_Nota_Enem",
                                                           "PROPORCAO_DE_CONCLUINTES_PARTICIPANTES_COM_NOTA_NO_ENEM as Prop_Concluintes_Participantes_Nota_Enem_IDD",
                                                           "AREA_DE_AVALIACAO as Curso")

In [14]:
dfIDDTratado.coalesce(1).write.option("header", True).csv("s3a://cf-murilo-pi-2022-01-sptech-bucket-tratados/idd", emptyValue='')


22/06/08 01:32:26 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/08 01:32:27 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
